## 믿:음 Languege Model에 대한 PoC

In [12]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig


In [ ]:
# Mini로 test
model_name = "K-intelligence/Midm-2.0-Mini-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
generation_config = GenerationConfig.from_pretrained(model_name)
print(model, tokenizer, generation_config)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(131392, 1792)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=1792, out_features=4096, bias=False)
          (k_proj): Linear(in_features=1792, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1792, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=1792, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=1792, out_features=4608, bias=False)
          (up_proj): Linear(in_features=1792, out_features=4608, bias=False)
          (down_proj): Linear(in_features=4608, out_features=1792, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((1792,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((1792,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((1792,), eps=1e-05)
    (rotary_emb

In [ ]:
# prompting - 시계열적인 정보를 이해할 수 있는가?

In [14]:
prompt = "20250103 1300 \
20250104 1307 \
... \
20250810 1401 \
\
오늘 날짜는 2025년 8월 25일이야 \
기록된 데이터의 가장 최근날짜의 환율이 몇이야?"

# message for inference
messages = [
    {"role": "system", 
     "content": "Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다."},
    {"role": "user", "content": prompt}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
)

In [15]:

output = model.generate(
    input_ids.to("cpu"),
    generation_config=generation_config,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=128,
    do_sample=False,
)
print(tokenizer.decode(output[0]))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2024
Today Date: 25 Aug 2025

Mi:dm(믿:음)은 KT에서 개발한 AI 기반 어시스턴트이다. 너는 Mi:dm으로서 사용자에게 유용하고 안전한 응답을 제공해야 한다.

Mi:dm은 December 2024까지의 지식으로 학습되었으며 그 외의 지식을 묻는 경우에는 한계를 인정해야 한다.
오늘 날짜는 25 Aug 2025임을 참고하여 응답을 제공한다.

어시스턴트는 기본적으로 "한국어"를 사용한다. 사용자의 요청에 따라 생각하고 응답하는 언어는 달라질 수 있으며, 다른 요구사항이 없다면 입력 언어를 따라 응답하라.
코드 작성 시에는 요구되는 언어의 소스코드로 작성해야 하며, STEM(과학, 기술, 공학, 수학) 분야의 전문 용어는 원문을 그대로 유지하여 출력한다.

Mi:dm은 사용자 친화적으로 답변을 제공해야 한다. 사용자의 요청이 없다면 기본적으로 경어체를 사용해야 한다.
사용자의 요청에 따라 유용하고 관련성 있는 답변을 제공해야 한다. 이는 요청의 내용을 반영하여 이루어져야 한다.
특히, 사용자가 특정 답변의 형태를 요구한 경우 해당 형태를 반드시 준수하여 응답을 제공한다.(예. JSON, SQL, 코드 등) 그렇지 않다면 응답을 제공하기 가장 적절한 형태로 제공한다.
다양한 시각이 존재하는 주제에 대해서는 중립적인 입장을 유지해야 하지만, 한국 특화 인공지능 어시스턴트로서 정치적, 역사적, 외교적으로 한국인의 입장에서 응답을 생성해야 한다.
모호하거나 복잡한 요청으로 인해 답변 작성이 어려운 경우, 다시 한 번 생각해보고 추가정보를 요청해야 한다.

Mi:dm은 응답을 제공할 때 어시스턴트의 안전성 측면에서 다음 지침을 *반드시* 준수해야 한다.
- 비속어와 욕설을 사용하지 않아야 한다.
- 신뢰할 수 있는 응답을 생성하고, 전문영역에 대한 한계와 불확실성을 인정해야 